#  Text Generation using GRU (Word-Level)

# Import Required Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Dataset

In [3]:
text = """artificial intelligence is transforming modern society.

it is used in healthcare finance education and transportation.
machine learning allows systems to improve automatically with experience.
data plays a critical role in training intelligent systems.
large datasets help models learn complex patterns.
deep learning uses multi layer neural networks.
neural networks are inspired by biological neurons.
each neuron processes input and produces an output.
training a neural network requires optimization techniques.
gradient descent minimizes the loss function.

natural language processing helps computers understand human language.
text generation is a key task in nlp.
language models predict the next word or character.
recurrent neural networks handle sequential data.
lstm and gru models address long term dependency problems.
however rnn based models are slow for long sequences.

transformer models changed the field of nlp.
they rely on self attention mechanisms.
attention allows the model to focus on relevant context.
transformers process data in parallel.
this makes training faster and more efficient.
modern language models are based on transformers.

education is being improved using artificial intelligence.
intelligent tutoring systems personalize learning.
automated grading saves time for teachers.
online education platforms use recommendation systems.
technology enhances the quality of learning experiences.

ethical considerations are important in artificial intelligence.
fairness transparency and accountability must be ensured.
ai systems should be designed responsibly.
data privacy and security are major concerns.
researchers continue to improve ai safety.

text generation models can create stories poems and articles.
they are used in chatbots virtual assistants and content creation.
generated text should be meaningful and coherent.
evaluation of text generation is challenging.
human judgement is often required.

continuous learning is essential in the field of ai.
research and innovation drive technological progress.
students should build strong foundations in mathematics.
programming skills are important for ai engineers.
practical experimentation enhances understanding.
"""

# Word-Level Tokenization

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1
print("Vocabulary size:", total_words)

# Create Input Sequences

In [ ]:
input_sequences = []

for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

max_seq_len = max(len(seq) for seq in input_sequences)
print("Maximum sequence length:", max_seq_len)

# Padding

In [ ]:
input_sequences = pad_sequences(
    input_sequences, maxlen=max_seq_len, padding='pre'
)

X = input_sequences[:, :-1]
y = input_sequences[:, -1]

y = tf.keras.utils.to_categorical(y, num_classes=total_words)

print("X shape:", X.shape)
print("y shape:", y.shape)

# Build GRU Model

In [ ]:
model = Sequential([
    Embedding(total_words, 100, input_length=max_seq_len - 1),

    GRU(256, return_sequences=True),
    Dropout(0.3),

    GRU(256),
    Dropout(0.3),

    Dense(256, activation='relu'),
    Dense(total_words, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# Training

In [ ]:
history = model.fit(
    X, y,
    epochs=50,
    batch_size=32,
    verbose=1
)

# Save the Model

In [ ]:
model.save("gru_word_text_generator.h5")
print("GRU model saved successfully!")

# Sampling Function

In [ ]:
def sample_with_temperature(preds, temperature=0.8):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

## Text Generation Function

In [ ]:
def generate_text(seed_text, next_words=30, temperature=0.7):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences(
            [token_list], maxlen=max_seq_len - 1, padding='pre'
        )

        preds = model.predict(token_list, verbose=0)[0]
        next_index = sample_with_temperature(preds, temperature)

        next_word = tokenizer.index_word.get(next_index, "")
        seed_text += " " + next_word

    return seed_text

## Run Generation

In [ ]:
seed = "artificial intelligence"
output = generate_text(seed, next_words=40, temperature=0.6)

print("Seed Text:", seed)
print("\nGenerated Text:\n")
print(output)

## Analysis

- GRU effectively captures long-term dependencies
- Uses fewer parameters than LSTM
- Trains faster while maintaining good accuracy
- Generated text is coherent and meaningful

## Conclusion

The GRU-based text generation model provides an efficient alternative to LSTM.
It produces coherent word-level text with reduced computational complexity,
making it suitable for real-time and resource-constrained applications.